# Getting Started with the AppEEARS API: Submitting and Downloading an Area Request
### This tutorial demonstrates how to use Python to connect to the AppEEARS API 
The Application for Extracting and Exploring Analysis Ready Samples ([AppEEARS](https://lpdaacsvc.cr.usgs.gov/appeears/)) offers a simple and efficient way to access and transform geospatial data from a variety of federal data archives in an easy-to-use web application interface. AppEEARS enables users to subset [geospatial data](https://lpdaacsvc.cr.usgs.gov/appeears/products) spatially, temporally, and by band/layer for point and area samples. AppEEARS returns not only the requested data, but also the associated quality values, and offers interactive visualizations with summary statistics in the web interface.  The [AppEEARS API](https://lpdaacsvc.cr.usgs.gov/appeears/api/) offers users **programmatic access** to all features available in AppEEARS, with the exception of visualizations. The API features are demonstrated in this notebook.  
***
### Example: Submit an area request using a U.S. National Park boundary as the region of interest for extracting elevation, vegetation and land surface temperature data    
Connect to the AppEEARS API, query the list of available products, submit an area sample request, download the request, become familiar with the AppEEARS Quality API, and import the results into Python for visualization. AppEEARS area sample requests allow users to subset their desired data by spatial area via vector polygons (shapefiles or GeoJSONs). Users can also reproject and reformat the output data. AppEEARS returns the valid data from the parameters defined within the sample request.       
#### Data Used in the Example:
- Data layers: 
    - NASA MeaSUREs Shuttle Radar Topography Mission (SRTM) Version 3 Digital Elevation Model  
        - [SRTMGL1.003](https://doi.org/10.5067/MEaSUREs/SRTM/SRTMGL1.003), 30m, static: 'Band1' 
    - Combined MODIS Leaf Area Index (LAI)  
        - [MCD15A3H.006](https://doi.org/10.5067/MODIS/MCD15A3H.006), 500m, 4 day: 'Lai_500m'      
    - Terra MODIS Land Surface Temperature    
        - [MOD11A2.006](https://doi.org/10.5067/MODIS/MOD11A2.006), 1000m, 8 day: 'LST_Day_1km', 'LST_Night_1km'     
***  
# Topics Covered:
1. **Getting Started**  
    1a. Enable Access to the API  
    1b. Set Up the Working Environment    
    1c. Login  
2. **Query Available Products [Product API]**  
    2a. Search and Explore Available Products [List Products]  
    2b. Search and Explore Available Layers [List Layers]      
3. **Submit an Area Request [Tasks]**  
    3a. Import a Shapefile  
    3b. Search and Explore Available Projections [Spatial API]   
    3c. Compile a JSON [Task Object]  
    3d. Submit a Task Request [Submit Task]  
    3e. Retrieve Task Status [Retrieve Task]  
4. **Download a Request [Bundle API]**    
    4a. Explore Files in Request Output [List Files]    
    4b. Download all Files in a Request [Download File]   
5. **Explore AppEEARS Quality Service [Quality API]**  
    5a. Explore Quality Layers [List Quality Layers]    
    5b. Show Quality Values [List Quality Values]   
    5c. Decode Quality Values [Decode Quality Values]  
6. **BONUS: Import Request Output and Visualize**    
    6a. Import a GeoTIFF     
    6b. Plot a GeoTIFF   
***
### Dependencies:
- This tutorial was tested using Python 3.6.1.  
- A [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to complete this tutorial. You can create an account at the link provided.    
- To execute section 6, the [Geospatial Data Abstraction Library](http://www.gdal.org/) (GDAL) is required. 
***
### AppEEARS Information:
To access AppEEARS, visit: https://lpdaacsvc.cr.usgs.gov/appeears/  
> For comprehensive documentation of the full functionality of the AppEEARS API, please see the AppEEARS [API Documentation](https://lpdaacsvc.cr.usgs.gov/appeears/api/) 


***
### Files Used in this Tutorial:  
- [Administrative Boundaries of National Park System Units 12/31/2017 - National Geospatial Data Asset (NGDA) NPS National Parks Dataset](https://irma.nps.gov/DataStore/DownloadFile/594958)  

### Source Code used to Generate this Tutorial: UPDATE LATER
- [Jupyter Notebook](https://git.earthdata.nasa.gov/projects/LPDUR/repos/cole-sandbox/browse/.ipynb_checkpoints/AppEEARS_GeoTIFF_TS-checkpoint.ipynb) 

***

# 1. Getting Started
<div class="alert alert-block alert-warning" >
<b>NOTE:</b> If this is your first time using the AppEEARS API, you must first enable API access by following the instructions provided below.</div> 

***
## 1a. Enable Access to the API
#### AppEEARS API access requires the same [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) as the AρρEEARS user interface. In addition to having a valid NASA Earthdata Login account, the API feature must be enabled for the user within AρρEEARS.
#### To enable access to the AppEEARS API, navigate to the [AppEEARS website](https://lpdaacsvc.cr.usgs.gov/appeears/). Click the *Sign In* button in the top right portion of the AppEEARS landing page screen.  
![AppEEARS Sign In](https://lpdaacsvc.cr.usgs.gov/assets/images/help/image001.73bb446d.png)  
#### Once you are signed in, click the *Manage User* icon in the top right portion of the AppEEARS landing page screen and select *Settings*. 
![Manage User -> Settings](https://lpdaacsvc.cr.usgs.gov/assets/images/help/api/image001.30a681c0.png)      
#### Select the *Enable API* box to gain access to the AppEEARS API.  
![Enable API Access](https://lpdaacsvc.cr.usgs.gov/assets/images/help/api/image002.2262e4d3.png)

***
## 1b. Set Up the Working Environment
#### Next, import the required packages, set the input/working directory, and create an output directory for the results.

In [ ]:
# Import packages
import requests as r
import getpass, pprint, time, os, cgi

#### If you are missing any of the packages above, download them in order to use the full functionality of this tutorial. 

In [ ]:
# Set input directory, change working directory
inDir = 'C:/Users/ckrehbiel/Documents/appeears-api-getting-started/'            # IMPORTANT: Update to dir on your OS
os.chdir(inDir)                                                                 # Change to working directory

# below, set the AppEEARS API to a variable (this URL also will take you to the AppEEARS API documentation)
api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'

<div class="alert alert-block alert-warning" >
<b>If you plan to execute this tutorial on your own OS, `inDir` above needs to be changed.</b>
</div> 

***
## 1c. Login
#### To submit a request, you must first login to the AppEEARS API. Use the `getpass` package to enter your NASA Earthdata login **Username** and **Password**. When prompted after executing the code block below, enter your username followed by your password.

In [ ]:
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      # Input NASA Earthdata Login Username
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')  # Input NASA Earthdata Login Password

#### Use the `requests` package to post your username and password. A successful login will provide you with a token to be used later in this tutorial to submit a request. For more information or if you are experiencing difficulties, please see the [API Documentation](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#login)

In [ ]:
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert base API URL, call the login service, provide credentials & return json
del user, password                                                           # Remove user and password information upon a successful login 
token_response                                                               # Print response

#### Above, you should see a Bearer token. Notice that this token will expire approximately 48 hours after being acquired. 
***
# 2. Query Available Products [[Product API](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#product)]
## 2a. Search and Explore Available Products [[List Products](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#list-products)]
#### The product API provides details about all of the products and layers available in AppEEARS. Below, call the product API to list all of the products available in AppEEARS.

In [ ]:
product_response = r.get('{}product'.format(api)).json()                         # request all products in the product service
print('AppEEARS currently supports {} products.'.format(len(product_response)))  # Print the number of products available in AppEEARS

#### Next, create a dictionary indexed by product name, making it easier to query a specific product.

In [ ]:
products = {p['ProductAndVersion']: p for p in product_response} # Create a dictionary indexed by the product name and version
products['MCD15A3H.006']                                         # Print information for MCD15A3H.006 LAI/FPAR Product

#### The product service provides many useful details, including if a product is currently available in AppEEARS, a description, and information on the spatial and temporal resolution. 

#### Below, make a list of all product+version names, and search for products containing *LAI* in their description. 

In [ ]:
prodNames = {p['ProductAndVersion'] for p in product_response} # Make list of all products (including version)

# Make a for loop to search list of products 'Description' for the acronym 'LAI'
for p in prodNames:                                            
    if 'Leaf Area Index' in products[p]['Description']:
        pprint.pprint(products[p])                             # Print the information for each product containing LAI in its description

#### Using the information above, start a list of desired products by using the highest temporal resolution LAI product available, `MCD15A3H.006`.  

In [ ]:
prods = ['MCD15A3H.006']     # Start a list for products to be requested, beginning with MCD15A3H.006
prods.append('MOD11A2.006')  # Append the MOD11A2.006 8 day LST product to the list of products desired
prods.append('SRTMGL1.003')  # Append the SRTMGL1.003 product to the list of products desired

***
# 2b. Search and Explore Available Layers [[List Layers](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#list-layers)]
#### The product API allows you to call all of the layers available for a given product. Each product is referenced by its `ProductAndVersion` property. For a list of the layer names only, print the keys from the dictionary above. 

In [ ]:
lst_response = r.get('{}product/{}'.format(api, prods[1])).json()  # request all layers for the second product (index 1) in the product list: MOD11A2.006
list(lst_response.keys())

#### Use the dictionary key `'LST_Day_1km'` to see the information for that layer in the response.

In [ ]:
lst_response['LST_Day_1km']                                                       # Print layer response

#### AppEEARS also allows subsetting data spectrally (by band). Create a tupled list with the product name and the specific layers desired. 

In [ ]:
layers = [(prods[1],'LST_Day_1km'),(prods[1],'LST_Night_1km')]  # Create tupled list linking desired product with desired layers

#### Next, request the layers for the `MCD15A3H.006` product.

In [ ]:
lai_response = r.get('{}product/{}'.format(api, prods[0])).json()  # request all layers for the first product (index 0) in the product list: MCD15A3H.006
list(lai_response.keys())                                          # Print the LAI layer names 

In [ ]:
lai_response['Lai_500m']['Description']  # Make sure the correct layer is requested

#### Above, `Lai_500m` is the desired layer within the `MCD15A3h.006` product. 
#### Next, append `Lai_500m` to the tupled list of desired product/layers. 

In [ ]:
layers.append((prods[0],'Lai_500m'))

#### Thirdly, request the layers for the `SRTMGL1.003` product.

In [ ]:
dem_response = r.get('{}product/{}'.format(api, prods[2])).json()  # request all layers for the third product (index 2) in the product list: MCD15A3H.006
list(dem_response.keys())                                          # Print the SRTM DEM layer names 

#### Finally, append `Band1` to the tupled list of desired products/layers.

In [ ]:
layers.append((prods[2],'Band1'))

#### Below, take the tupled list (layers) and create a dictionary to store each layer+product combination. This will make it easier to insert into the json file used to submit a request in Section 3. 

In [ ]:
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
prodLayer

***
# 3. Submit an Area Request [[Tasks](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#tasks)]
#### The **Submit task** API call provides a way to submit a new request to be processed. It can accept data via JSON, query string, or a combination of both. In the example below, compile a json and submit a request. Tasks in AppEEARS correspond to each request associated with your user account. Therefore, each of the calls to this service requires an authentication token (see Section 1C.), which is stored in a header below. 

In [ ]:
token = token_response['token']                      # Save login token to a variable
head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request below

---
## 3a. Import a Shapefile
#### In this section, begin by importing a shapefile using the `geopandas` package. The shapefile is publically available for download from the [NPS website](https://irma.nps.gov/DataStore/Reference/Profile/2224545?lnv=True).

In [ ]:
import geopandas as gpd
import json

# Read in shapefile
nps = gpd.read_file('{}nps_boundary.shp'.format(inDir + os.sep)) # UPDATE
print(nps.head())                                                              # Print first few lines of df

#### Below, query the geopandas dataframe for the national park that you are interested in using for your region of interest, here *Grand Canyon National Park*.

In [ ]:
nps_gc = nps[nps['UNIT_NAME']=='Grand Canyon National Park'].to_json() # Extract GCNP and set to variable
nps_gc = json.loads(nps_gc)                                            # Convert to json format

---
## 3b. Search and Explore Available Projections [[Spatial API](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#spatial)]   
#### The spatial API provides some helper services used to support submitting area task requests. The call below will retrieve the list of supported projections in AppEEARS.

In [ ]:
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
projections                                               # Print projections and information

#### Create a dictionary of projections with projection `Name` as the keys.

In [ ]:
projs = {}                                  # Create an empty dictionary
for p in projections: projs[p['Name']] = p  # Fill dictionary with `Name` as keys
list(projs.keys())                          # Print dictionary keys

#### Print the information for the projection used in this tutorial, Geographic.

In [ ]:
projs['geographic']

---
## 3c. Compile a JSON [[Task Object](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#task-object)]
#### In this section, begin by setting up the information needed to compile an acceptable json for submitting an AppEEARS point request. For detailed information on required json parameters, see the [API Documentation](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#tasks). 

In [ ]:
task_name = input('Enter a Task Name: ') # user-defined name of the task 'NPS Vegetation Area' 

In [ ]:
task_type = ['point','area']        # Type of task, area or point
proj = projs['geographic']['Name']  # Set output projection 
outFormat = ['geotiff', 'netcdf4']  # Set output file format type
startDate = '07-01-2017'            # Start of the date range for  which to extract data: MM-DD-YYYY
endDate = '07-31-2017'              # End of the date range for  which to extract data: MM-DD-YYYY
recurring = False                   # Specify True for a recurring date range
# yearRange = [2000, 2016] if recurring = True, set yearRange and change start and end date to MM-DD

#### Below, compile the JSON to be submitted as a point request. Notice that `nps_gc` is inserted from the shapefile transformed to a json via the `geopandas` and `json` packages above in section 3a. 

In [ ]:
task = {
    'task_type': task_type[1],
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat[0]}, 
                         'projection': proj},
         'geo': nps_gc,
    }
}

***
## 3b. Submit a Task Request [[Submit Task](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#submit-task)]
#### Below, post a call to the API task service, using the `task` json created above.

In [ ]:
task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post the json to the API task service, and return the response as a json
task_response                                                                 # Print task response

***
## 3c. Retrieve Task Status [[Retrieve Task](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#retrieve-task)]
#### This API call will list all of the requests associated with your user account, automatically sorted by date descending with the most recent requests listed first. 

#### The AppEEARS API contains some helpful formatting resources. Below, limit the API response to 4 and set `pretty` to True to format the response as an organized json, making it easier to read. Additional information on AppEEARS API [Pagination](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#pagination) and [formatting](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#formatting) can be found in the API documentation. 

In [ ]:
params = {'limit': 4, 'pretty': True} # Limit API response to 4 most recent entries, and return as pretty json

In [ ]:
tasks_response = r.get('{}task'.format(api),params = params, headers=head).json() # Query the task service setting the params and header containing user token
tasks_response                                                                    # Print tasks response

#### Next, take the task id returned from the `task_response` that was generated when submitting your request, and use the AppEEARS API status service to check the status of your request.

In [ ]:
task_id = '7dffacbc-0ac3-41d9-a453-fa62918b00a2'#task_response['task_id']                                               # Set task id from request submission
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID and user credentials, return json
#status_response                                                                  # Print response

#### Above, if your request is still processing, you can find information on the status of how close it is to completing. 
#### Below, call the task service for your request every 20 seconds to check the status of your request.

In [ ]:
# Ping API until request is complete, then continue to Section 4
starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(20.0 - ((time.time() - starttime) % 20.0))
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

***
# 4. Download a Request [[Bundle API](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#bundle)]
#### Before downloading the request output, set up an output directory to store the output files, and examine the files contained in the request output. 

In [ ]:
destDir = os.path.join(inDir, task_name)                # Set up output directory using input directory and task name
if not os.path.exists(destDir):os.makedirs(destDir)     # Create the output directory

---
## 4a. Explore Files in Request Output [[List Files](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#list-files)]
#### The bundle API provides information about completed tasks. For any completed task, a bundle can be queried to return the files contained as a part of the task request. Below, call the bundle API and return all of the output files.

In [ ]:
bundle = r.get('{}bundle/{}'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json
bundle                                                    # Print bundle contents

***
## 4b. Download Files in a Request (Automation) [[Download File](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#download-file)]
#### Now, use the contents of the bundle to select the file name and id and store as a dictionary. 

In [ ]:
files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
files    

#### Use the `files` dictionary and a for loop to automate downloading all of the output files into the output directory. 

In [ ]:
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f),stream=True)                                 # Get a stream to the bundle file
    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])  # Parse the filename from the Content-Disposition header 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))

***
# 5. Explore AppEEARS Quality Service [[Quality API](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#quality)]
#### The quality API provides quality details about all of the data products available in AppEEARS. Below are examples of how to query the quality API for listing quality products, layers, and values. The final example (Section 5c.) demonstrates how AppEEARS quality services can be leveraged to decode pertinent quality values for your data.
#### First, reset pagination to include `offset` which allows you to set the number of results to skip before starting to return information. Next, make a call to list all of the data product layers and where the associated quality product and layer information is located.

In [ ]:
params = {'limit': 6, 'pretty': True, 'offset': 20}                     # Limit response to 6 most recent entries, start w/ 20th entry, and return pretty json
quality_response = r.get('{}quality'.format(api), params=params).json() # Call quality API using pagination and return json
quality_response

---
## 5a. Explore Quality Layers [[List Quality Layers](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#list-quality-layers)]
#### This API call will list all of the quality layer information for a product.

In [ ]:
product = 'MCD15A3H.006'                                        # Product used in the example
ql_response = r.get('{}quality/{}'.format(api,product)).json()  # Call API to retrieve quality layers for selected product
ql_response                                                     # Print response

---
## 5b. Show Quality Values [[List Quality Values](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#list-quality-values)]
#### This API call will list all of the values for a given quality layer.

In [ ]:
qlayer = ql_response[1]['QualityLayers'][0]                                # Set quality layer from ql_response for 'Lai_500m' above
qv_response = r.get('{}quality/{}/{}'.format(api, product, qlayer)).json()  # Call API for list of list of bit-word quality values
qv_response                                                                # Print response

---
## 5c. Decode Quality Values [[Decode Quality Values](https://lpdaacsvc.cr.usgs.gov/appeears/api/?language=Python%203#decode-quality-values)]
#### This API call will decode the bits for a given quality value.

In [ ]:
val = 1                                                                            # Set a specific value
q_response = r.get('{}quality/{}/{}/{}'.format(api, product, qlayer, val)).json()  # Call quality API for specific value
q_response                                                                         # Print Response

***
# 6. BONUS: Import Request Output and Visualize
#### Here, import one of the output GeoTIFFs and show some basic visualizations using `matplotlib`.

In [ ]:
# Import packages
import matplotlib.pyplot as plt
import numpy as np
from osgeo import gdal
list(files.values())  # List files downloaded

---
## 6a. Import a GeoTIFF
#### To perform the next step below, you will need to have [GDAL](http://www.gdal.org/) installed on your OS. Open the GeoTIFF file for the SRTM DEM, and read in as an array.


In [ ]:
destDir = 'C:/Users/ckrehbiel/Documents/appeears-api-getting-started/NPS Vegetation Area/'

In [ ]:
dem = gdal.Open(destDir + '/SRTMGL1.003_Band1_doy2000042_aid0001.tif' ) # Read file in
demBand = dem.GetRasterBand(1)                                          # Read the band (layer)
demData = demBand.ReadAsArray().astype('float')                         # Import band as an array with type float

#### Next, query the metadata for the fill value, and set fill value equal to `nan`.

In [ ]:
demFill = demBand.GetNoDataValue()            # Returns fill value
demData[demData == demFill] = np.nan          # Set fill value to nan

---
## 6b. Plot a GeoTIFF
#### In this section, begin by highlighting the functionality of the matplotlib plotting package.  

In [ ]:
# Set matplotlib plots inline
%matplotlib inline 

#### First, make a basic plot of the DEM data.

In [ ]:
plt.imshow(demData);  # Visualize a basic plot of the DEM data

#### Next, add some additional parameters to the plot.

In [ ]:
fig = plt.figure(figsize = (10,7.5))  # Set the figure size (x,y)
plt.axis('off')                       # Remove the axes' values
ax = fig.add_subplot(111)             

# Plot the array, using a colormap and setting a custom linear stretch based on the min/max Elevation values
plt.imshow(demData, vmin = np.nanmin(demData), vmax = np.nanmax(demData), cmap = 'terrain');

#### Finally, add important map items including a legend and title.

In [ ]:
plt.style.use("dark_background")    # Default to a black background
fig2 = plt.figure(figsize=(10,7.5))  # Set the figure size
plt.axis('off')                     # Remove the axes' values
ax1 = fig2.add_subplot(111)           # Make a subplot
fig2.subplots_adjust(top=3.8)        # Adjust spacing

# Add a title to the figure
ax1.set_title('SRTM DEM: Grand Canyon NP', fontsize=15, fontweight='bold', color = 'white')


# Plot the masked data, using a colormap and setting a custom linear stretch based on the min/max DEM values
im = plt.imshow(demData, vmin = np.nanmin(demData), vmax = np.nanmax(demData), cmap = 'terrain');
# Add a colormap legend
cb = plt.colorbar(im, orientation='horizontal', fraction=0.047, pad=0.004, shrink=0.6)
cb.set_label(label='Elevation (m)', color = 'white')  # Set Label and color
cb.outline.set_edgecolor('white')                     # Set edge color


### This example can provide a template to use for your own research workflows. Leveraging the AppEEARS API for searching, extracting, and formatting analysis ready data, and importing it directly into Python means that you can keep your entire research workflow in a single software program, from start to finish. 

***
<div class="alert alert-block alert-info">
<h1> Contact Information </h1> 
<h3> Material written by Cole Krehbiel$^{1}$ </h3>
<b>Contact:</b> LPDAAC@usgs.gov  
<b>Voice:</b> +1-605-594-6116  
<b>Organization:</b> Land Processes Distributed Active Archive Center (LP DAAC)  
<b>Website:</b> https://lpdaac.usgs.gov/  
<b>Date last modified:</b> 03-30-2018  

$^{1}$Innovate! Inc., contractor to the U.S. Geological Survey, Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, 57198-001, USA. Work performed under USGS contract G15PD00467 for LP DAAC$^{2}$.

$^{2}$LP DAAC Work performed under NASA contract NNG14HH33I. 